### Load ensemble pathway and Assign highest frequency associated with each node to the respective node 

In [1]:
import pandas as pd

replace the ensemble pathway path for 05 or 060 and uncomment to_csv code cell to save

In [3]:
ensemble_pathway_path = "C:/Users/sumed/BDS-SROP-24/spras-hiv-results/hiv-benchmarking-05-060-modified-v2/hiv060-ml/ensemble-pathway.txt"
ensemble_pathway_df = pd.read_csv(ensemble_pathway_path, sep='\t')
ensemble_pathway_df.head()

,Node1,Node2,Frequency,Direction
0,1433B_HUMAN,FRYL_HUMAN,0.04,U
1,1433B_HUMAN,L7RRS6_HUMAN,0.04,U
2,1433B_HUMAN,PI3R4_HUMAN,0.04,U
3,1433B_HUMAN,WEE1_HUMAN,0.04,U
4,1433E_HUMAN,HDAC4_HUMAN,0.04,U


In [4]:
ensemble_pathway_df['Frequency'].max()

0.33

In [5]:
node1_freq = ensemble_pathway_df.drop(columns = ['Node2', 'Direction'])
node1_freq.head()

,Node1,Frequency
0,1433B_HUMAN,0.04
1,1433B_HUMAN,0.04
2,1433B_HUMAN,0.04
3,1433B_HUMAN,0.04
4,1433E_HUMAN,0.04


In [6]:
node2_freq = ensemble_pathway_df.drop(columns = ['Node1', 'Direction'])
node2_freq.head()

,Node2,Frequency
0,FRYL_HUMAN,0.04
1,L7RRS6_HUMAN,0.04
2,PI3R4_HUMAN,0.04
3,WEE1_HUMAN,0.04
4,HDAC4_HUMAN,0.04


In [7]:
max_node1_freq = node1_freq.groupby(['Node1']).max().reset_index()
max_node1_freq.rename(columns = {'Frequency': 'Freq1'}, inplace = True)
max_node1_freq.head()

,Node1,Freq1
0,1433B_HUMAN,0.04
1,1433E_HUMAN,0.04
2,1433G_HUMAN,0.04
3,1433T_HUMAN,0.04
4,2A5A_HUMAN,0.08


In [8]:
max_node2_freq = node2_freq.groupby(['Node2']).max().reset_index()
max_node2_freq.rename(columns = {'Frequency': 'Freq2'}, inplace = True)
max_node2_freq.head()

,Node2,Freq2
0,2A5D_HUMAN,0.04
1,2AAB_HUMAN,0.04
2,3BP5L_HUMAN,0.04
3,4EBP1_HUMAN,0.04
4,4ET_HUMAN,0.04


In [9]:
node_df_merged = max_node1_freq.merge(max_node2_freq, left_on='Node1', right_on='Node2', how='outer')
node_df_merged.head()

,Node1,Freq1,Node2,Freq2
0,1433B_HUMAN,0.04,NaN,NaN
1,1433E_HUMAN,0.04,NaN,NaN
2,1433G_HUMAN,0.04,NaN,NaN
3,1433T_HUMAN,0.04,NaN,NaN
4,2A5A_HUMAN,0.08,NaN,NaN


In [10]:
def select_max_freq_and_node(row):
    max_freq = 0
    node = ""
    if pd.isna(row['Node2']) and pd.isna(row['Freq2']):
        max_freq = row['Freq1']
        node = row['Node1']
    elif pd.isna(row['Node1']) and pd.isna(row['Freq1']):
        max_freq = row['Freq2']
        node = row['Node2']
    else:
        max_freq = max(row['Freq1'], row['Freq2'])
        node = row['Node1']
    return node, max_freq

In [11]:
node_df_merged[['Node', 'max_freq']] = node_df_merged.apply(select_max_freq_and_node, axis=1, result_type='expand')
node_df_merged.drop(columns = ['Node1', 'Node2', 'Freq1', 'Freq2'], inplace = True)
node_df_merged.head()

,Node,max_freq
0,1433B_HUMAN,0.04
1,1433E_HUMAN,0.04
2,1433G_HUMAN,0.04
3,1433T_HUMAN,0.04
4,2A5A_HUMAN,0.08


In [12]:
# node_df_merged.to_csv("C:/Users/sumed/BDS-SROP-24/spras_processed_data/hiv05-max-node-freq-ensemble.csv", index=False)